In [ ]:
# Libraries that are part of the Python standard library
import gzip
import itertools
import json
import pathlib
import string
import time

# Third-party packages
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
run_nltk_downloader = False

if run_nltk_downloader:
    nltk.download()

In [ ]:
# create a pathlib.Path object for the data
raw_data_filepath = pathlib.Path('relevant_data_location.json')

# open the data filepath
with open(raw_data_filepath, 'r') as fp:
    tweet_data = json.load(fp)
    
print(f'The data includes {len(tweet_data)} observations.')

In [ ]:
#np.random.seed(0)
#sample_size = 5000
#excerpt = np.random.choice(tweet_data, size=sample_size, replace=False,)
excerpt=tweet_data[0:5000]
column_names = [
    'Datetime',
    'Username',
    'User',
    'Loc1',
    'Loc2',
    'Language',
    'Text',
]
#tweet_df = pd.DataFrame(tweet_data, columns = column_names)
exc_tweet_df = pd.DataFrame(excerpt, 
                        columns = column_names)

In [ ]:
user_defined_stop_words = ['..', '...', 'get', '\u200d', '’', '“', '”']

my_mwe_tuples = [
    ('u', '.', 's', '.'),
    ('covid', '-', '19'),
]

i = nltk.corpus.stopwords.words('english')
j = list(string.punctuation) + user_defined_stop_words
stopwords = set(i).union(j)

tknzr = nltk.tokenize.TweetTokenizer()
mwe_tknzr = nltk.tokenize.MWETokenizer()
mwe_tknzr.add_mwe(my_mwe_tuples)
lemmatizer = nltk.stem.WordNetLemmatizer()
stemmer = nltk.stem.porter.PorterStemmer()


def create_tok(x):
    tokens = tknzr.tokenize(x.lower())
    tokens = [token for token in tokens if token not in string.punctuation]
    tokens = [token for token in tokens if token not in set(stopwords)]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens
    

In [ ]:
exc_tweet_df['Tokens'] = exc_tweet_df['Text'].apply(create_tok)

In [ ]:
#filter to get just tweets from AL use on 'tweet_data' for full dataset
#test = exc_tweet_df.loc[exc_tweet_df['Loc2'].str.contains("AL"),:]
exc_tweet_df.head()

In [ ]:
#convert Datetime column to datetime format to allow for easier sorting/more flexible manipulation
test.loc[:,'Datetime'] = pd.to_datetime(test['Datetime'], 
                                      format = '%a %b %d %H:%M:%S %z %Y')

In [ ]:
test.head()